(1) Data Description

The data analysis will use two files, players.csv (196 observations, 7 columns) and sessions.csv (1,535 observations, 5 columns).

players.csv - one row per player, and columns include gender, age, hours played and experience
sessions.csv - one row per play session, and columns include start and end times

players.csv variables:
| variable | type | description |
|----------|------|-------------|
| experience | character | experience level (Beginner, Amateur, Regular, Pro, Veteran) |
| subscribe | logical | newsletter subscription (True/False) |
| hashedEmail | character | player identifier |
| player_hours | double | total hours played |
| name | character | player name |
| gender | character | player's gender |
| Age | double | age in years |

sessions.csv variables:
| variable | type | description |
|----------|------|-------------|
| hashedEmail | character | player identifier |
| start_time | character | session start time |
| end_time | character | session end time |
| original_start_time | double | raw start time |
| original_end_time | double | raw end time |

(2) Questions

Broad question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

Specific question: How might a player's experience and total hours played predict their tendency to subscribe to a newsletter, and how does this differ between players of various ages